In [1]:
import os 


In [2]:
pwd%


'c:\\Users\\amenm\\OneDrive\\Desktop\\Predecting_BTC_Price\\Bitcoin_predection_price\\research'

In [3]:
os.chdir("../")

In [4]:
pwd%


'c:\\Users\\amenm\\OneDrive\\Desktop\\Predecting_BTC_Price\\Bitcoin_predection_price'

lazem nzid l2_reg w nchouf kifech bich naddi il shape 

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    full_model_path:Path
    shape_path:Path
    lstm_units_1:int 
    lstm_units_2 :int 
    dropout_rate_1:float
    dropout_rate_2:float
    l2_reg_1:float
    l2_reg_2:float


In [6]:
from src.LSTM_BTC_Prediction.constants  import *
from src.LSTM_BTC_Prediction.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath) 
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config=self.config.prepare_base_model

        create_directories([config.root_dir])  

        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir= config.root_dir,
            full_model_path = config.full_model_path,
            shape_path = config.shape_path,
            lstm_units_1= self.params.LSTM_UNITS_1,
            lstm_units_2 = self.params.LSTM_UNITS_2,
            dropout_rate_1= self.params.DROPOUT_RATE_1,
            dropout_rate_2 = self.params.DROPOUT_RATE_2 ,
            l2_reg_1=self.params.L2_REG_1,
            l2_reg_2=self.params.L2_REG_2  )
        
        return prepare_base_model_config

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.regularizers import L2
from src.LSTM_BTC_Prediction import logger 
import numpy as np 


In [9]:
class PrepareBaseModel:

    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_shape(self):
        try:
            self.trainX = np.load(self.config.shape_path) 
            logger.info("Downloading trainX successful")
        except Exception as e:
            logger.error(f"Error occurred during downloading trainX: {e}")

    def get_base_model(self):
        try:
            self.get_shape()  # Call get_shape to ensure trainX is loaded before building the model

            self.model = Sequential()
            self.model.add(Bidirectional(LSTM(units=self.config.lstm_units_1, return_sequences=True, kernel_regularizer=L2(self.config.l2_reg_1)),
                                         input_shape=(self.trainX.shape[1], self.trainX.shape[2])))
            self.model.add(Dropout(self.config.dropout_rate_1))
            self.model.add(Bidirectional(LSTM(units=self.config.lstm_units_2, kernel_regularizer=L2(self.config.l2_reg_2))))
            self.model.add(Dropout(self.config.dropout_rate_2))
            self.model.add(Dense(units=1))
            self.model.summary()
            logger.info("Compiling model completed")
            self.save_model(model=self.model, path=self.config.full_model_path)
            logger.info('model saved')
        except Exception as e:
            logger.error(f"Error occurred during building the base model: {e}")

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_shape()
    prepare_base_model.get_base_model() 
except Exception as e:
    raise e

[2024-05-06 15:19:16,475: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-06 15:19:16,479: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-06 15:19:16,481: INFO: common: created directory at: artifacts]
[2024-05-06 15:19:16,482: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-05-06 15:19:16,499: INFO: 330417884: Downloading trainX successful]
[2024-05-06 15:19:16,509: INFO: 330417884: Downloading trainX successful]


c:\Users\amenm\miniconda3\envs\lstmenv\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 120, 312)       │       212,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120, 312)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 248)            │       433,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 248)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           249 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 645,913 (2.46 MB)

 Trainable params: 645,913 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

[2024-05-06 15:19:16,631: INFO: 330417884: Compiling model completed]
[2024-05-06 15:19:16,632: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2024-05-06 15:19:16,650: INFO: 330417884: model saved]
